<a href="https://colab.research.google.com/github/mehurlock94/identifying-factors-contributing-to-armed-conflict/blob/main/Identifying_factors_contributing_to_armed_conflict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import six

import plotly.express as px

from google.colab import files

# Import csvs as dataframes
df_fuel = pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/Fuel_exports_percent_merchandise_exports.csv')
df_energy = pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/Energy_Use.csv')
df_male_ag = pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/Percent_Male_Agriculture.csv')
df_electricity_access = pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/Percent_access_to_electricity.csv')
df_low_income = pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/Income_share_by_lowest_20.csv')
df_school_enroll = pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/Percent_school_enrollment.csv')
df_literacy = pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/Literacy_rate.csv')
df_high_tech=pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/High_tech_exports.csv')
df_deaths = pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/conflict_deaths.csv')
df_conflicts = pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/conflicts.csv')
df_imports=pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/Arms_imports.csv')
df_exports=pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/Arms_exports.csv')
df_expense_gdp=pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/Military_expenditure_GDP.csv')
df_expense_capita=pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/Military_expenditure_capita.csv')
df_regions=pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/countries_regions.csv')
df_regions=df_regions.rename(columns={'Unnamed: 1':'Country'})
df_fuel_import_plot=pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/fuel_imports_data.csv')
df_lowinc_conflict_plot=pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/lowinc_conflicts_plot_data.csv')
df_various_expenditure_plot=pd.read_csv('https://raw.githubusercontent.com/mehurlock94/identifying-factors-contributing-to-armed-conflict/main/various_expenditure_capita.csv')

# This is old code that identifies the years with the most data points for each df
# length_tracker={}
# counter=0
# for df in df_list:
#   for column in df:
#     length_tracker[str(counter)+'_'+column]=df[column].isnull().sum(axis=0)
#   counter+=1

# best_dict={}
# counter=0
# for df in df_list:
#   lowest=5000
#   for key in length_tracker:
#     if str(counter)+'_' in key and 'Country' not in key and 'ame' not in key and 'ode' not in key:
#       if length_tracker[key]<lowest:
#         lowest=length_tracker[key]
#         best = key
#   best_dict[best]=lowest
#   counter+=1

# print (df_fuel.isnull().sum(axis=0))
# # Lists to select specific dataframes and eventual column headers
# df_list =[df_fuel]
# master_labels = ['Fuel']

# Lists which can be varied to select data to be exported in the final csv. Years should reflect those which maximize the number of points for a particular data type.
df_list=[df_fuel, df_male_ag, df_electricity_access, df_school_enroll,df_low_income]
df_list_columns=['Fuel','Male_ag','Electricity','School_enroll','Low_income']
df_list_years=[2010, 2012, 2012, 2018, 2018, 2005]

# Pull unique country names from the lists to identify differences and clean
deaths_conflicts=pd.unique(df_deaths['Country'])
imports_exports=pd.unique(df_imports['Country'])
expense=pd.unique(df_expense_gdp['Country'])
merge_list=pd.unique(df_fuel['Country Name'])

# Create a dataframe that merges key datapoints for analysis
merged=pd.DataFrame(merge_list, columns=['Country'])
counter=0
for df, year in zip(df_list, df_list_years):
  df=df.rename(columns={'Country Name':'Country'})
  df.drop(df.columns.difference(['Country',str(year)]),1,inplace=True)
  merged=merged.merge(df, how='inner', on='Country')
  df_regions=df_regions.merge(df, on='Country')
  merged=merged.rename(columns={str(year):df_list_columns[counter]})
  df_regions=df_regions.rename(columns={str(year):df_list_columns[counter]})
  counter+=1

# Clean the data further by replacing Nan values with the average for a given region. This average comes from the regions defined in the SIPRI dataset
"""This section of code should only be run if you want Nan values to be filled with the mean for that region"""
regions=pd.unique(df_regions['Region'])
headers=list(merged.columns.values)
df_medians=pd.DataFrame(index=regions, columns=headers[1:])
for region in regions:
  runner=df_regions[df_regions['Region']==region]
  for header in headers[1:]:
    median=runner[header].mean()
    df_medians.at[region,header]=median
df_medians.reset_index(inplace=True)
df_medians=df_medians.rename(columns={'index':'Region'})
df_regions=df_regions.merge(df_medians, on='Region')

# Further clean the data to remove duplicate column names post-merge
df_regions.drop(df_regions.columns.difference(['Region','Country','Fuel_y','Male_ag_y', 'Electricity_y', 'School_enroll_y', 'Low_income_y']),1,inplace=True)
df_regions=df_regions.rename(columns={'Fuel_y':'Fuel','Male_ag_y':'Male_ag', 'Electricity_y':'Electricity', 'School_enroll_y':'School_enroll', 'Low_income_y':'Low_income'})
merged=merged.set_index('Country')  
df_regions=df_regions.set_index('Country')
merged=merged.fillna(df_regions)

# Define a function to produce csv outputs for the various dataframes. Mainly to avoid the need to re-write excessively
def merger(df_list, columns_list,labels_list, merged_df):
  counter=0
  for df, column in zip(df_list, columns_list):
    df.drop(df.columns.difference(['Country',str(column)]),1,inplace=True)
    df=df.rename(columns={str(column):labels_list[counter]})
    runner=merged_df.merge(df, how='inner', on='Country')
    runner=runner.dropna()
    runner.to_csv(str(labels_list[counter])+'.csv')
    files.download(str(labels_list[counter])+'.csv')
    counter+=1
    return runner

# Produce data output csv files
merger([df_deaths],[2019],['Deaths'],merged)
merger([df_conflicts],[2008],['Conflicts'],merged)
merger([df_imports],[2019],['Imports'],merged)
merger([df_exports],[2006],['Exports'],merged)
merger([df_expense_capita],[2018],['Expense_Capita'],merged)
merger([df_expense_gdp],[2018],['Expense_GPD'],merged)
merger([df_imports, df_exports, df_expense_capita, df_expense_gdp],[2019, 2006, 2018, 2018],['Imports','Exports','Expense_Capita','Expense_GDP'],merged)

# Generate plots of fuel exports vs arms imports
joined=pd.melt(df_fuel_import_plot, id_vars=['Fuel'], value_vars=['Imports'])
graph=px.scatter(joined, x='Fuel', y='value',title='Predictability of Fuel on Armed Conflicts', labels={'Fuel':'Fuel Exports','value':'Arms Imports'})
graph.write_html('Fuel_Imports.html')
# files.download('Fuel_Imports.html')

# Generate plots of low income percentage vs armed conflicts
joined=pd.melt(df_lowinc_conflict_plot, id_vars=['Low_income'], value_vars=['Conflicts'])
graph=px.scatter(joined, x='Low_income', y='value',title='Effect of Lower Income Population on Armed Conflicts', labels={'Low_income':'Income Shared by Lowest 20%','value':'Armed Conflicts'})
graph.write_html('Lowinc_conflicts.html')
# files.download('Lowinc_conflicts.html')

# Generate plots of fuel exports on military expenditure
joined=pd.melt(df_various_expenditure_plot, id_vars=['Fuel'], value_vars=['Expense_Capita'])
graph=px.scatter(joined, x='Fuel', y='value',title='Effect of Fuel Exports on Military Spending per Capita', labels={'Fuel':'Fuel Exports','value':'Military Expenses per Capita'})
graph.write_html('Fuel_percapita.html')
# files.download('Fuel_percapita.html')

# Generate plots of percent males in agriculture vs military expenditure
joined=pd.melt(df_various_expenditure_plot, id_vars=['Male_ag'], value_vars=['Expense_Capita'])
graph=px.scatter(joined, x='Male_ag', y='value',title='Effect of Farming Population on Military Spending per Capita', labels={'Male_ag':'% Males in Agriculture','value':'Military Expenses per Capita'})
graph.write_html('Maleag_percapita.html')
# files.download('Maleag_percapita.html')

# Generate plots of percent electricity availability vs military expenditure
joined=pd.melt(df_various_expenditure_plot, id_vars=['Electricity'], value_vars=['Expense_Capita'])
graph=px.scatter(joined, x='Electricity', y='value',title='Effect of Electricity Availability on Military Spending per Capita', labels={'Electricity':'% Population with Electricity','value':'Military Expenses per Capita'})
graph.write_html('Electricity_percapita.html')
# files.download('Electricity_percapita.html')

# Generate plots of academic enrollment vs military expenditure
joined=pd.melt(df_various_expenditure_plot, id_vars=['School_enroll'], value_vars=['Expense_Capita'])
graph=px.scatter(joined, x='School_enroll', y='value',title='Effect of School Enrollment on Military Spending per Capita', labels={'School_enroll':'Ratio of Girls:Boys Enrolled','value':'Military Expenses per Capita'})
graph.write_html('School_percapita.html')
files.download('School_percapita.html')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>